In [2]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import random

import pyblp
import numpy as np
import pandas as pd

pyblp.options.digits = 2
pyblp.options.verbose = False
pyblp.__version__

from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('../data/processed_data/merged_characteristics.csv')
keys = ['HIOS ID', 'IssuerId', 'County', 'State', 'FIPS County Code', 'Policy County FIPS Code','County Name']


#create market data...
df['shares'] = df['Ever Enrolled Count']/df['DP03_0095E']

#add blp columns
df['prices'] = df['EHBPercentTotalPremium']
df['product_ids'] = df['IssuerId'].astype(str) +  df['County'].astype(str) 
df['market_ids'] = df['County']
df['firm_ids'] = df['IssuerId']

#demand_instrument0
MktIds = np.array(pd.get_dummies(df['IssuerId']))
MktIds2 = (MktIds.T).dot(MktIds)
dummies_proj = MktIds.dot( np.linalg.inv( MktIds2 ) ).dot( MktIds.T )
df['demand_instruments0'] = dummies_proj.dot(df['prices']) #average price across markets


#fix problematic columns =
df = df.rename(columns={'Average Monthly Advanced CSR Payment for Consumers with 94%':'csr_pay_94',
                  'Average Monthly Advanced CSR Payment for Consumers with 87%':'csr_pay_87',
                       'Total Number of Consumers':'csr_tot',
                        'Number of Consumers with CSR AV of 94%':'csr_tot_94'})

#create mkt share data
blp_keys = ['market_ids', 'product_ids', 'firm_ids', 'shares', 'prices','demand_instruments0']

nonlinear_cols = ['csr_pay_94']
#['act_value','csr_pay_94','csr_pay_87','SBCHavingDiabetesCopayment','SBCHavingaBabyCoinsurance']

linear_cols = [ 'act_value', 'MetalLevel_Silver']
#['CompositeRatingOffered_No','MetalLevel_Silver','MetalLevel_Platinum',
# 'BeginPrimaryCareCostSharingAfterNumberOfVisits' , 'csr_tot', 'csr_tot_94', 'PlanType_Indemnity']            
#'CSRVariationTypeBinary','DentalOnlyPlan_No','CompositeRatingOffered_No',

firm_cols =  linear_cols+ nonlinear_cols    
mkt_cols = [ 'DP05_0015PE' ,'DP05_0072PE', 'DP03_0029PE','DP05_0069PE']


blp_data = df[blp_keys+linear_cols+nonlinear_cols+mkt_cols]


#standardize the cols
for col in firm_cols:
    #this line is causing a warning?
    blp_data[col] = blp_data[col]/blp_data[col].var()

#zero shares are not okay?
blp_data = blp_data[blp_data['shares']!=0]

<ipython-input-3-05c51e36a581>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  blp_data[col] = blp_data[col]/blp_data[col].var()


In [4]:
#split into training and test
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(blp_data, blp_data, test_size=test_size)

#set_up product/agent data
product_data = blp_data[blp_keys+linear_cols+nonlinear_cols]
product_data['market_ids'] = product_data['market_ids'].astype(str)

#setup agent data
agent_data = blp_data[['market_ids']+ mkt_cols]
agent_data = agent_data.groupby('market_ids',as_index=False).mean()
agent_data['market_ids'] = agent_data['market_ids'].astype(str)

#double check its the same
#market_ids = extract_matrix(agent_data, 'market_ids')
#pr

<ipython-input-4-74369b12f966>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_data['market_ids'] = product_data['market_ids'].astype(str)


In [9]:
# do the estimation with just product characteristics?
X1_formula = ''
for i in linear_cols:
    X1_formula = X1_formula +  i + ' + '
X1_formula = '1 + prices + ' + X1_formula[:-3]


X2_formula = ''
for i in nonlinear_cols:
    X2_formula = X2_formula + i + ' + '
X2_formula = '0 + prices + ' + X2_formula[:-3]


X1_formulation = pyblp.Formulation(X1_formula)
X2_formulation = pyblp.Formulation(X2_formula)
product_formulations = (X1_formulation, X2_formulation)

mc_integration = pyblp.Integration('monte_carlo', size=50, specification_options={'seed': 0})

pr_integration = pyblp.Integration('product', size=5)


mc_problem = pyblp.Problem(product_formulations, product_data, integration=mc_integration)


bfgs = pyblp.Optimization('bfgs', {'gtol': 1e-2})

In [10]:
results1 = mc_problem.solve(sigma=np.ones((len(nonlinear_cols)+1, len(nonlinear_cols)+1)), optimization=bfgs, method='1s')
results1

In [11]:
#add agents to the problem?
agent_formula = ''
for i in mkt_cols:
    agent_formula = agent_formula + i + ' + '
agent_formula = '0 + ' + agent_formula[:-3]


agent_formulation = pyblp.Formulation(agent_formula)
agent_formulation


DP05_0015PE + DP05_0072PE + DP03_0029PE + DP05_0069PE

In [22]:
mc_integration = pyblp.Integration('monte_carlo', size=1, specification_options={'seed': 0})

full_problem = pyblp.Problem(
    product_formulations,
    product_data,
    agent_formulation,
    agent_data, 
    integration=mc_integration
)


bfgs = pyblp.Optimization('bfgs', {'gtol': 1e-2})


full_results = full_problem.solve(pi = np.zeros( (len(nonlinear_cols)+1,len(mkt_cols)) ), 
                                  sigma=np.ones((len(nonlinear_cols)+1, len(nonlinear_cols)+1)),
                                  optimization=bfgs)

full_results

KeyboardInterrupt: 